# Sample for using transformer with KFServing SDK 

The notebook shows how to use KFServing SDK to create KFService with transformer, predictor.

In [29]:
from kubernetes import client

from kfserving import KFServingClient
from kfserving import constants
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2TransformerSpec
from kfserving import V1alpha2PyTorchSpec
from kfserving import V1alpha2CustomSpec
from kfserving import V1alpha2KFServiceSpec
from kfserving import V1alpha2KFService
from kubernetes.client import V1Container
from kubernetes.client import V1ResourceRequirements

## Define KFService with Transformer

Add predictor and transformer on the endpoint spec

In [30]:
api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
default_endpoint_spec = V1alpha2EndpointSpec(
                          predictor=V1alpha2PredictorSpec(
                            pytorch=V1alpha2PyTorchSpec(
                              storage_uri='gs://kfserving-samples/models/pytorch/cifar10',
                              model_class_name= "Net",
                              resources=V1ResourceRequirements(
                                  requests={'cpu':'100m','memory':'1Gi'},
                                  limits={'cpu':'100m', 'memory':'1Gi'}))),
                          transformer=V1alpha2TransformerSpec(
                            custom=V1alpha2CustomSpec(
                              container=V1Container(
                              image='yuzisun/image-transformer:latest',
                              name='user-container',
                              resources=V1ResourceRequirements(
                                  requests={'cpu':'100m','memory':'1Gi'},
                                  limits={'cpu':'100m', 'memory':'1Gi'})))))
    
kfsvc = V1alpha2KFService(api_version=api_version,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(
                              name='cifar10', namespace='kubeflow'),
                          spec=V1alpha2KFServiceSpec(default=default_endpoint_spec))

## Create KFService with Transformer

Call KFServingClient to create KFService.

In [31]:
KFServing = KFServingClient()
KFServing.create(kfsvc)

{'apiVersion': 'serving.kubeflow.org/v1alpha2',
 'kind': 'KFService',
 'metadata': {'creationTimestamp': '2019-09-23T08:11:00Z',
  'generation': 1,
  'name': 'cifar10',
  'namespace': 'kubeflow',
  'resourceVersion': '1882645',
  'selfLink': '/apis/serving.kubeflow.org/v1alpha2/namespaces/kubeflow/kfservices/cifar10',
  'uid': 'ada803a9-ddd9-11e9-adc5-42010a80027c'},
 'spec': {'default': {'predictor': {'pytorch': {'modelClassName': 'Net',
     'resources': {'limits': {'cpu': '100m', 'memory': '1Gi'},
      'requests': {'cpu': '100m', 'memory': '1Gi'}},
     'runtimeVersion': 'latest',
     'storageUri': 'gs://kfserving-samples/models/pytorch/cifar10'}},
   'transformer': {'custom': {'container': {'image': 'yuzisun/image-transformer:latest',
      'name': 'user-container',
      'resources': {'limits': {'cpu': '100m', 'memory': '1Gi'},
       'requests': {'cpu': '100m', 'memory': '1Gi'}}}}}}},
 'status': {'canary': {}, 'default': {}}}

## Check the KFService

In [32]:
KFServing.get('cifar10', namespace='kubeflow', watch=True, timeout_seconds=120)

NAME                 READY      DEFAULT_TRAFFIC CANARY_TRAFFIC  URL                                               
cifar10              Unknown                                    http://cifar10-predict.kubeflow.example.com       
cifar10              Unknown                                    http://cifar10-predict.kubeflow.example.com       
cifar10              Unknown                                    http://cifar10-predict.kubeflow.example.com       
cifar10              Unknown                                    http://cifar10-predict.kubeflow.example.com       
cifar10              True                                       http://cifar10-predict.kubeflow.example.com       


## Predict the image

In [33]:
CLUSTER_IP=!kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
!MODEL_NAME=cifar10
%env CLUSTER_IP=$CLUSTER_IP

env: CLUSTER_IP=['34.67.11.82']


In [34]:
import torch
import torchvision
import io
import base64
from PIL import Image
import numpy as np

testset = torchvision.datasets.CIFAR10(root='../../../../docs/samples/pytorch/data', train=False)
imgByteArr = io.BytesIO()
testset[0][0].save(imgByteArr, format='PNG')
imgByteArr = imgByteArr.getvalue()
b64=base64.b64encode(imgByteArr).decode("utf-8")
byte_array = base64.b64decode(b64)
image = Image.open(io.BytesIO(byte_array))
a = np.asarray(image)
im = Image.fromarray(a)

In [35]:
import os
url = "http://" + os.environ['CLUSTER_IP'].strip("[]''") + "/v1/models/cifar10:predict"
headers = { 'Host': 'cifar10-predict.kubeflow.example.com' }
import requests 
import json
instances = [{'image_bytes': {'b64': b64}}]
response = requests.post(url, json.dumps({'instances': instances}), headers=headers)
response.content

b'{"predictions": [[-1.6099601984024048, -2.6461076736450195, 0.32844462990760803, 2.4825074672698975, 0.43524616956710815, 2.3108043670654297, 1.00056791305542, -0.4232763648033142, -0.5100948214530945, -1.7978394031524658]]}'

## Delete the KFService

In [9]:
KFServing.delete('cifar10', namespace='kubeflow')

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'cifar10',
  'group': 'serving.kubeflow.org',
  'kind': 'kfservices',
  'uid': '39cccc35-ddcf-11e9-adc5-42010a80027c'}}